In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import initializers
import tensorflow as tf

In [18]:
train = pd.read_csv('train.csv')

# Data Preprocessing
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan , strategy='median')
Age_2 = train['Age'].values.reshape(1,-1)
Age_2 = Age_2.T
Age_2 = imputer.fit_transform(Age_2)
train['Age_2'] = Age_2
train.head()

test = pd.read_csv('test.csv')

Age_test = test['Age'].values.reshape(1,-1)
Age_test = Age_test.T
Age_test = imputer.fit_transform(Age_test)
test['Age_2'] = Age_test
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_2
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,34.5
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,47.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,62.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,27.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,22.0


In [19]:
train['Embarked'].value_counts() # As we see that 'S' value is very frequent, we replace the null with 'S'
train.Embarked.fillna('S',inplace = True)

In [20]:
test['Fare'].fillna(test['Fare'].mean(),inplace = True)

In [21]:
train.drop(['Age','Cabin'], inplace = True, axis = 1)
test.drop(['Age','Cabin'], inplace = True, axis = 1)

In [22]:
train = pd.get_dummies(train,columns  =['Sex','Embarked','Pclass'], drop_first = True)
test = pd.get_dummies(test,columns  =['Sex','Embarked','Pclass'], drop_first = True)
train.head()

,PassengerId,Survived,Name,SibSp,Parch,Ticket,Fare,Age_2,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",1,0,A/5 21171,7.2500,22.0,1,0,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,0,PC 17599,71.2833,38.0,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",0,0,STON/O2. 3101282,7.9250,26.0,0,0,1,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,0,113803,53.1000,35.0,0,0,1,0,0
4,5,0,"Allen, Mr. William Henry",0,0,373450,8.0500,35.0,1,0,1,0,1


In [23]:
train_target = train.iloc[:,1].values
train_predictors = train.iloc[:,6:].values
test_predictors = test.iloc[:,5:].values


#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_predictors = sc.fit_transform(train_predictors)
test_predictors = sc.transform(test_predictors)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(train_predictors, train_target, test_size = 0.2, random_state = 0)
n_cols = len(X_train)
n_cols

712

In [30]:
model = Sequential()
model.add(Dense(50,activation = tf.keras.layers.LeakyReLU(alpha= 0.3) ,kernel_initializer = initializers.he_uniform(),input_shape=(7,),activity_regularizer=regularizers.l1(1e-4)))#Hidden layer 1

model.add(Dense(25,activation = tf.keras.layers.LeakyReLU(alpha= 0.3),kernel_initializer = initializers.he_uniform(),activity_regularizer=regularizers.l1(1e-4))) # Hidden layer 2
model.add(Dropout(0.5))
model.add(Dense(10,activation = tf.keras.layers.LeakyReLU(alpha=0.3))) # Hidden layer 3

model.add(Dense(1,activation='sigmoid')) # output layer

model.compile(optimizer = 'adam', loss ='binary_crossentropy',metrics = ['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, 
        verbose=1, mode='auto', restore_best_weights=True)


model.fit(X_train,y_train, validation_data = (X_test,y_test), epochs = 500, batch_size = 712)

Epoch 1/500
1/1 [==============================] - 0s 171ms/step - loss: 0.7177 - accuracy: 0.5829 - val_loss: 0.6500 - val_accuracy: 0.6313
Epoch 2/500
1/1 [==============================] - 0s 40ms/step - loss: 0.7217 - accuracy: 0.5955 - val_loss: 0.6364 - val_accuracy: 0.6480
Epoch 3/500
1/1 [==============================] - 0s 40ms/step - loss: 0.6915 - accuracy: 0.6208 - val_loss: 0.6237 - val_accuracy: 0.6480
Epoch 4/500
1/1 [==============================] - 0s 36ms/step - loss: 0.6975 - accuracy: 0.6320 - val_loss: 0.6118 - val_accuracy: 0.6704
Epoch 5/500
1/1 [==============================] - 0s 47ms/step - loss: 0.6650 - accuracy: 0.6629 - val_loss: 0.6004 - val_accuracy: 0.6872
Epoch 6/500
1/1 [==============================] - 0s 41ms/step - loss: 0.6693 - accuracy: 0.6475 - val_loss: 0.5893 - val_accuracy: 0.7151
Epoch 7/500
1/1 [==============================] - 0s 38ms/step - loss: 0.6698 - accuracy: 0.6194 - val_loss: 0.5789 - val_accuracy: 0.7151
Epoch 8/500
1/1 [==

Epoch 59/500
1/1 [==============================] - 0s 42ms/step - loss: 0.4934 - accuracy: 0.7739 - val_loss: 0.4163 - val_accuracy: 0.7877
Epoch 60/500
1/1 [==============================] - 0s 40ms/step - loss: 0.4845 - accuracy: 0.7781 - val_loss: 0.4158 - val_accuracy: 0.7877
Epoch 61/500
1/1 [==============================] - 0s 45ms/step - loss: 0.5030 - accuracy: 0.7865 - val_loss: 0.4154 - val_accuracy: 0.7877
Epoch 62/500
1/1 [==============================] - 0s 217ms/step - loss: 0.5070 - accuracy: 0.7753 - val_loss: 0.4150 - val_accuracy: 0.7877
Epoch 63/500
1/1 [==============================] - 0s 47ms/step - loss: 0.4909 - accuracy: 0.7893 - val_loss: 0.4146 - val_accuracy: 0.7877
Epoch 64/500
1/1 [==============================] - 0s 38ms/step - loss: 0.4896 - accuracy: 0.7921 - val_loss: 0.4142 - val_accuracy: 0.7933
Epoch 65/500
1/1 [==============================] - 0s 42ms/step - loss: 0.4881 - accuracy: 0.7963 - val_loss: 0.4138 - val_accuracy: 0.7989
Epoch 66/500

Epoch 117/500
1/1 [==============================] - 0s 46ms/step - loss: 0.4598 - accuracy: 0.8160 - val_loss: 0.4029 - val_accuracy: 0.8156
Epoch 118/500
1/1 [==============================] - 0s 40ms/step - loss: 0.4546 - accuracy: 0.8034 - val_loss: 0.4031 - val_accuracy: 0.8156
Epoch 119/500
1/1 [==============================] - 0s 53ms/step - loss: 0.4640 - accuracy: 0.8104 - val_loss: 0.4033 - val_accuracy: 0.8156
Epoch 120/500
1/1 [==============================] - 0s 38ms/step - loss: 0.4563 - accuracy: 0.8062 - val_loss: 0.4034 - val_accuracy: 0.8156
Epoch 121/500
1/1 [==============================] - 0s 39ms/step - loss: 0.4581 - accuracy: 0.8034 - val_loss: 0.4034 - val_accuracy: 0.8156
Epoch 122/500
1/1 [==============================] - 0s 47ms/step - loss: 0.4627 - accuracy: 0.8006 - val_loss: 0.4035 - val_accuracy: 0.8156
Epoch 123/500
1/1 [==============================] - 0s 39ms/step - loss: 0.4423 - accuracy: 0.8160 - val_loss: 0.4035 - val_accuracy: 0.8156
Epoch 

Epoch 175/500
1/1 [==============================] - 0s 44ms/step - loss: 0.4483 - accuracy: 0.8118 - val_loss: 0.4034 - val_accuracy: 0.8101
Epoch 176/500
1/1 [==============================] - 0s 36ms/step - loss: 0.4303 - accuracy: 0.8132 - val_loss: 0.4036 - val_accuracy: 0.8101
Epoch 177/500
1/1 [==============================] - 0s 45ms/step - loss: 0.4288 - accuracy: 0.8216 - val_loss: 0.4036 - val_accuracy: 0.8101
Epoch 178/500
1/1 [==============================] - 0s 38ms/step - loss: 0.4519 - accuracy: 0.8104 - val_loss: 0.4038 - val_accuracy: 0.8101
Epoch 179/500
1/1 [==============================] - 0s 40ms/step - loss: 0.4552 - accuracy: 0.8118 - val_loss: 0.4039 - val_accuracy: 0.8101
Epoch 180/500
1/1 [==============================] - 0s 43ms/step - loss: 0.4427 - accuracy: 0.7992 - val_loss: 0.4040 - val_accuracy: 0.8101
Epoch 181/500
1/1 [==============================] - 0s 46ms/step - loss: 0.4451 - accuracy: 0.8090 - val_loss: 0.4041 - val_accuracy: 0.8101
Epoch 

1/1 [==============================] - 0s 39ms/step - loss: 0.4456 - accuracy: 0.8076 - val_loss: 0.4049 - val_accuracy: 0.8101
Epoch 233/500
1/1 [==============================] - 0s 38ms/step - loss: 0.4390 - accuracy: 0.8132 - val_loss: 0.4048 - val_accuracy: 0.8101
Epoch 234/500
1/1 [==============================] - 0s 38ms/step - loss: 0.4457 - accuracy: 0.8104 - val_loss: 0.4047 - val_accuracy: 0.8101
Epoch 235/500
1/1 [==============================] - 0s 45ms/step - loss: 0.4436 - accuracy: 0.8160 - val_loss: 0.4046 - val_accuracy: 0.8101
Epoch 236/500
1/1 [==============================] - 0s 45ms/step - loss: 0.4513 - accuracy: 0.8174 - val_loss: 0.4045 - val_accuracy: 0.8101
Epoch 237/500
1/1 [==============================] - 0s 40ms/step - loss: 0.4433 - accuracy: 0.8188 - val_loss: 0.4044 - val_accuracy: 0.8101
Epoch 238/500
1/1 [==============================] - 0s 37ms/step - loss: 0.4292 - accuracy: 0.8301 - val_loss: 0.4044 - val_accuracy: 0.8101
Epoch 239/500
1/1 [=

Epoch 290/500
1/1 [==============================] - 0s 46ms/step - loss: 0.4220 - accuracy: 0.8188 - val_loss: 0.4077 - val_accuracy: 0.8156
Epoch 291/500
1/1 [==============================] - 0s 42ms/step - loss: 0.4348 - accuracy: 0.8174 - val_loss: 0.4079 - val_accuracy: 0.8156
Epoch 292/500
1/1 [==============================] - 0s 46ms/step - loss: 0.4455 - accuracy: 0.8118 - val_loss: 0.4081 - val_accuracy: 0.8156
Epoch 293/500
1/1 [==============================] - 0s 61ms/step - loss: 0.4230 - accuracy: 0.8202 - val_loss: 0.4082 - val_accuracy: 0.8156
Epoch 294/500
1/1 [==============================] - 0s 47ms/step - loss: 0.4277 - accuracy: 0.8188 - val_loss: 0.4083 - val_accuracy: 0.8156
Epoch 295/500
1/1 [==============================] - 0s 43ms/step - loss: 0.4390 - accuracy: 0.8174 - val_loss: 0.4082 - val_accuracy: 0.8156
Epoch 296/500
1/1 [==============================] - 0s 51ms/step - loss: 0.4272 - accuracy: 0.8244 - val_loss: 0.4079 - val_accuracy: 0.8156
Epoch 

Epoch 348/500
1/1 [==============================] - 0s 43ms/step - loss: 0.4202 - accuracy: 0.8287 - val_loss: 0.4103 - val_accuracy: 0.8101
Epoch 349/500
1/1 [==============================] - 0s 45ms/step - loss: 0.4314 - accuracy: 0.8216 - val_loss: 0.4102 - val_accuracy: 0.8101
Epoch 350/500
1/1 [==============================] - 0s 43ms/step - loss: 0.4322 - accuracy: 0.8244 - val_loss: 0.4101 - val_accuracy: 0.8101
Epoch 351/500
1/1 [==============================] - 0s 55ms/step - loss: 0.4255 - accuracy: 0.8160 - val_loss: 0.4099 - val_accuracy: 0.8101
Epoch 352/500
1/1 [==============================] - 0s 43ms/step - loss: 0.4223 - accuracy: 0.8118 - val_loss: 0.4097 - val_accuracy: 0.8101
Epoch 353/500
1/1 [==============================] - 0s 44ms/step - loss: 0.4305 - accuracy: 0.8202 - val_loss: 0.4097 - val_accuracy: 0.8101
Epoch 354/500
1/1 [==============================] - 0s 45ms/step - loss: 0.4393 - accuracy: 0.8160 - val_loss: 0.4097 - val_accuracy: 0.8101
Epoch 

Epoch 406/500
1/1 [==============================] - 0s 39ms/step - loss: 0.4385 - accuracy: 0.8188 - val_loss: 0.4094 - val_accuracy: 0.8156
Epoch 407/500
1/1 [==============================] - 0s 39ms/step - loss: 0.4266 - accuracy: 0.8258 - val_loss: 0.4091 - val_accuracy: 0.8156
Epoch 408/500
1/1 [==============================] - 0s 39ms/step - loss: 0.4250 - accuracy: 0.8216 - val_loss: 0.4090 - val_accuracy: 0.8156
Epoch 409/500
1/1 [==============================] - 0s 47ms/step - loss: 0.4234 - accuracy: 0.8216 - val_loss: 0.4089 - val_accuracy: 0.8156
Epoch 410/500
1/1 [==============================] - 0s 38ms/step - loss: 0.4206 - accuracy: 0.8146 - val_loss: 0.4089 - val_accuracy: 0.8156
Epoch 411/500
1/1 [==============================] - 0s 37ms/step - loss: 0.4232 - accuracy: 0.8258 - val_loss: 0.4089 - val_accuracy: 0.8156
Epoch 412/500
1/1 [==============================] - 0s 37ms/step - loss: 0.4261 - accuracy: 0.8216 - val_loss: 0.4088 - val_accuracy: 0.8156
Epoch 

Epoch 464/500
1/1 [==============================] - 0s 36ms/step - loss: 0.4244 - accuracy: 0.8258 - val_loss: 0.4108 - val_accuracy: 0.8156
Epoch 465/500
1/1 [==============================] - 0s 39ms/step - loss: 0.4301 - accuracy: 0.8160 - val_loss: 0.4108 - val_accuracy: 0.8156
Epoch 466/500
1/1 [==============================] - 0s 35ms/step - loss: 0.4224 - accuracy: 0.8315 - val_loss: 0.4108 - val_accuracy: 0.8156
Epoch 467/500
1/1 [==============================] - 0s 47ms/step - loss: 0.4098 - accuracy: 0.8258 - val_loss: 0.4109 - val_accuracy: 0.8156
Epoch 468/500
1/1 [==============================] - 0s 40ms/step - loss: 0.4120 - accuracy: 0.8287 - val_loss: 0.4111 - val_accuracy: 0.8156
Epoch 469/500
1/1 [==============================] - 0s 40ms/step - loss: 0.4248 - accuracy: 0.8258 - val_loss: 0.4111 - val_accuracy: 0.8156
Epoch 470/500
1/1 [==============================] - 0s 37ms/step - loss: 0.4264 - accuracy: 0.8244 - val_loss: 0.4110 - val_accuracy: 0.8156
Epoch 

In [77]:
y_pred = model.predict(X_test).round()

In [78]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[104   6]
 [ 26  43]]


In [79]:
survived = model.predict(test_predictors).round()

In [80]:
test['Survived'] = survived
test['Survived'] = test['Survived'].astype(int)

In [60]:
test.head()

,PassengerId,Name,SibSp,Parch,Ticket,Fare,Age_2,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3,Survived
0,892,"Kelly, Mr. James",0,0,330911,7.8292,34.5,1,1,0,0,1,0
1,893,"Wilkes, Mrs. James (Ellen Needs)",1,0,363272,7.0000,47.0,0,0,1,0,1,0
2,894,"Myles, Mr. Thomas Francis",0,0,240276,9.6875,62.0,1,1,0,1,0,0
3,895,"Wirz, Mr. Albert",0,0,315154,8.6625,27.0,1,0,1,0,1,0
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,1,3101298,12.2875,22.0,0,0,1,0,1,1


In [82]:
Final = test.drop(test.columns.difference(['PassengerId','Survived']), axis = 1)

In [83]:
Final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [84]:
Final.to_csv('Titanic_with DeepLearning_3.csv', index=False)